In [37]:
import cv2
import os
import base64
import requests
import pickle
import numpy as np

url = "http://localhost:8080/api/gethog"
Band = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce','Swift', 'Tata Safari', 'Toyota Innova']


In [38]:
def img_HOG(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_data = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_data),"'")[1]
    response = requests.get(url, json={"img":data})
    
    return response.json()

In [39]:
#Read Data
def Read_Data(path):
    response = []
    for sub in os.listdir(path):
        for fn in os.listdir(path + '/' + sub):
            File_Img_Name = path + '/' + sub + "/" + fn
            img = cv2.imread(File_Img_Name)
            res = img_HOG(img)
            hog = list(res["HOG"])
            hog.append(Band.index(sub))
            response.append(hog)
    return response

In [41]:
# save data to pkl function
def Save_Pkl(FileName ,path):
    cars = Read_Data(path)
        
    write_path = FileName + ".pkl"
    pickle.dump(cars, open(write_path,"wb"))
    print("Data Preparation Is Done")


In [46]:
# Load Data to pkl
def Download_Pkl(FileName):
    dataset = pickle.load(open(FileName + '.pkl','rb'))
    return dataset

In [56]:
#เรียกใช้งาน Function SavePkl
Data_train = r'Data\train'
Save_Pkl('Data_train',Data_train)

Data Preparation Is Done


In [57]:
#เรียกใช้ function savePkl
Data_test = r'Data\test'
Save_Pkl('Data_test',Data_test)

Data Preparation Is Done


In [58]:
Data_train_car = Download_Pkl('Data_train')
print("Data train : ",len(Data_train_car))
Data_test_car = Download_Pkl('Data_test')
print("Data test : ",len(Data_test_car))

Data train :  3352
Data test :  813


In [59]:
# แปลง list ข้อมูล train เป็น array
Data_array_train = np.array(Data_train_car)
x_train = Data_array_train[:,0:-1]
y_train = Data_array_train[:,-1]


In [60]:
# แปลง list ข้อมูล test เป็น array
Data_array_test = np.array(Data_test_car)
x_test = Data_array_test[:,0:-1]
y_test = Data_array_test[:,-1]

In [53]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [61]:
# เทรนข้อมูลด้วย Decision Tree
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [62]:
# Data test
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 48.708487084870846
Confusion Matrix : 
 [[ 77  17  11  22  25  26  21]
 [  8  41   2   6   1   3   6]
 [ 15   4  27   6   3   8  12]
 [ 20   8   5  11   7  10  13]
 [ 11  11   6   6  49   2  17]
 [  8   3   1   9   8  69   8]
 [ 25  14   6   8  10   5 122]]


In [55]:
# save model file ด้วย pickle
Path_model ='CarBrand_model.pkl'
pickle.dump(clf, open(Path_model,'wb'))